# Cálculo y análisis de la tasa de deforestación

In [21]:
#python3
# Hans D. Escobar H. (hdescobarh@unal.edu.co)

import pandas as pd
import numpy as np
import os
import sys
sys.path.insert(0, os.path.abspath('../..'))
from landecology import deforestation_rate as dr

In [22]:
print(dr.DeforestationCalculator.__doc__)


    Collection of methods for determining forest coverture change. Equations are based on:

    Puyravaud, J.-P. (2003). Standardizing the calculation of the annual rate of deforestation.
    Forest Ecology and Management, 177(1–3), 593–596. https://doi.org/10.1016/S0378-1127(02)00335-3

    Raises:
        Exception: requested invalid formula
        NotImplementedError: requested method not implemented yet
    


## Datos

In [23]:
deforestation_df = pd.read_csv("./example_data_set.csv", index_col=0)
deforestation_df

,area_bosque_m2,area_pastos_m2,area_nubes_m2,area_sindatos_m2
año,,,,
2000,16149333600,6367233600,433800,77598000
2010,14814712800,7660449000,41839200,77598000
2020,13526236800,8754103800,236660400,77598000


## Resultados

### Tasa de cambio anual del bosque (Puyravaud)

In [24]:
# Define analysis parameters

coberture_keyname = "area_bosque_m2"
years = sorted([2010, 2000])
parameters: dict[str, float] = {
            "area_t1": deforestation_df.loc[years[0]][coberture_keyname],
            "area_t2": deforestation_df.loc[years[1]][coberture_keyname],
            "year_t1": years[0],
            "year_t2": years[1],
        }
rate_type = dr.DeforestationFormula.FOREST_CHANGE_PUYRAVAUD

In [25]:
# Perform analysis
forest_change = dr.DeforestationCalculator.calculate_deforestation_rate(rate_type, **parameters)
print("La tasa de cambio de bosque (basado en {}) es: {:.2f}%".format(rate_type.value, forest_change*100))

La tasa de cambio de bosque (basado en Puyravaud) es: -0.86%


In [26]:
rate_type_2 = dr.DeforestationFormula.FOREST_CHANGE_FAO
forest_change_2 = dr.DeforestationCalculator.calculate_deforestation_rate(rate_type_2, **parameters)
print("La tasa de cambio de bosque (basado en {}) es: {:.2f}%".format(rate_type_2.value, forest_change_2*100))

La tasa de cambio de bosque (basado en FAO) es: -0.86%


### Proyección para el 2020

La tasa de cambio propuesta por Puyravaud (2003) es equivalente a lo que en ecología de poblaciones se denomina "tasa intrínseca de crecimiento r". r se asume constante en el tiempo, y bajo el modelo exponencial:

$A_t = A_0 \cdot e^{r(t-t_0)}$

In [27]:
m2_to_ha2 = lambda area_m2: area_m2 / 10000
actual_area_2020_m2 = deforestation_df.loc[2020][coberture_keyname]
inferred_area_2020_m2 = (lambda t: parameters["area_t1"] * np.power(np.e, forest_change * t))(2020 - parameters["year_t1"])
print("Area medida: {:.0f} ha²\nArea proyectada: {:.0f} ha²".format(m2_to_ha2(actual_area_2020_m2), m2_to_ha2(inferred_area_2020_m2)))
absolute_error_m2 = inferred_area_2020_m2 - actual_area_2020_m2
absolute_error_ha2 = m2_to_ha2(absolute_error_m2)
percent_error = np.abs(absolute_error_m2/actual_area_2020_m2)*100
print("Error absoluto: {:.0f} ha²\nError porcentual: {:.2f} %".format(absolute_error_ha2,percent_error))

Area medida: 1352624 ha²
Area proyectada: 1359039 ha²
Error absoluto: 6415 ha²
Error porcentual: 0.47 %


## Referencias

Puyravaud, J.-P. (2003). Standardizing the calculation of the annual rate of deforestation. Forest Ecology and Management, 177(1–3), 593–596. https://doi.org/10.1016/S0378-1127(02)00335-3